# Predicting whether an order should be sent to a technical approver

For updates on the way Sagemaker or AWS behave compared to the notebook code, please refer to https://livebook.manning.com/#!/book/machine-learning-for-business/chapter-2/v-5/67

## Part 1: Load and examine the data

In [ ]:
#Hinterlegen der Bucket credentials

data_bucket = 'ie-mlforbusiness-01'
subfolder = 'ch02'
dataset = 'orders_with_predicted_value.csv'

In [ ]:
#Importieren von Bibliotheken

import pandas as pd
from time import sleep

import boto3
import sagemaker
import s3fs
from sklearn.model_selection import train_test_split

#Erstellen einer Rolle, um Sagemaker Ressourcen zu nutzen
role = sagemaker.get_execution_role()

#Herstellen einer Verbindung zu S3
s3 = s3fs.S3FileSystem(anon=False)

In [ ]:
#Einlesen des datensatzes aus S3 in einen Pnadas DataFrame
df = pd.___(f's3://{data_bucket}/{subfolder}/{dataset}')
df.head()

In [ ]:
#Identifikation von Bestellungen mit und ohne Approval

print(f'Number of rows in dataset: {df.shape[0]}')
print(df[df.columns[0]].value_counts())

## Part 2: Get the data into the right shape

In [ ]:
#Encoding der Daten mittels One-Hot Encoding

encoded_data = pd.get_dummies(df)
encoded_data.head()

In [ ]:
#Bestimmung der Korrelation mittels Features und Zielvariable
#Anzeigen der Features mit einer Korrelation > 10%

corrs = encoded_data.corr()['tech_approval_required'].abs()
columns = corrs[corrs > .1].index
corrs = corrs.filter(columns)
corrs

In [ ]:
#Weiterhin nutzen wir nur die Features, die einen Korrelation > 10 % aufweisen

encoded_data = encoded_data[columns]
encoded_data.head()

## Part 3: Create training, validation and test data sets

In [ ]:
#Erstellen von Trainings-, Test- und Validierungsdatensatz

#Training 70%, Val 20%, Test 10%
train_df, val_and_test_data = train_test_split(encoded_data, test_size=0.3, random_state=0)
val_df, test_df = train_test_split(val_and_test_data, test_size=0.333, random_state=0)

#Umwandeln der DataFrames in CSV und Speicherung in S3
train_data = train_df.to_csv(None, header=False, index=False).encode()
val_data = val_df.to_csv(None, header=False, index=False).encode()
test_data = test_df.to_csv(None, header=True, index=False).encode()


with s3.open(f'{data_bucket}/{subfolder}/processed/train.csv', 'wb') as f:
    f.write(train_data)

with s3.open(f'{data_bucket}/{subfolder}/processed/val.csv', 'wb') as f:
    f.write(val_data) 
    
with s3.open(f'{data_bucket}/{subfolder}/processed/test.csv', 'wb') as f:
    f.write(test_data) 
    
#Load data in sagemaker
train_input = sagemaker.TrainingInput(s3_data=f's3://{data_bucket}/{subfolder}/processed/train.csv', content_type='csv') # s3_input
val_input = sagemaker.TrainingInput(s3_data=f's3://{data_bucket}/{subfolder}/processed/val.csv', content_type='csv')    # s3_input

## Part 4: Train the model

In [ ]:
#Store SageMaker Session
sess = sagemaker.Session()

#Definition eines Containers, in dem AWS das Modell speichern kann
container = sagemaker.amazon.amazon_estimator.get_image_uri(
                boto3.Session().region_name,
                'xgboost',
                'latest')

#Erstellen des Modells
estimator = sagemaker.estimator.Estimator(
                container,
                role,
                train_instance_count=1, 
                train_instance_type='ml.m4.xlarge',#Server-Typ = Umfang vorhandener Ressourcen
                output_path=f's3://{data_bucket}/{subfolder}/output', #Speichert Output in S3
                sagemaker_session=sess)

#Definition von Hyperparametern für das Modell
estimator.set_hyperparameters(
                max_depth=5, #max Tiefe eines Baumes, je tiefer desto komplexer (Overfitting)
                subsample=0.7, # 30% der Daten bleiben für das einzelne Model ungenutzt
                objective='binary:logistic', #logistische Regression für binäre Klassifikation
                eval_metric = 'auc', # HP tunen sodass bester AUC-Wert erreicht wird
                num_round=100, # Anzahl der Trainingsrunden
                early_stopping_rounds=10) # Anzahl der Trainingsrunden nach denen das Training ohne Verbesserung beendet wird

#Trainieren/Fitten des Modells
estimator.fit({'train': train_input, 'validation': val_input})

## Part 5: Host the model

In [ ]:
#Erstellen eines Endpoints über den später auf das Modell zugegriffen werden kann

endpoint_name = 'order-approval'
try:
    sess.delete_endpoint(endpoint_name)
    sess.delete_endpoint_config(endpoint_name)
    print('Warning: Existing endpoint deleted to make way for your new endpoint.')
    sleep(30)
except:
    pass    

In [ ]:
#Bereitstellen eines Servers, um das Modell zu hosten
predictor = estimator.deploy(initial_instance_count=1,
               instance_type='ml.t2.medium',
               endpoint_name=endpoint_name
                            )

In [ ]:
#Ermöglicht predictor CSV Formate einfacher zu handhaben

from sagemaker.predictor import csv_serializer, json_serializer
from sagemaker.deserializers import JSONDeserializer   
from sagemaker.serializers import CSVSerializer        
#predictor.content_type = 'text/csv'
predictor.__setattr__(predictor.content_type, "text/csv")
predictor.serializer = CSVSerializer()      #csv_serializer
predictor.deserializer = JSONDeserializer() #None

## Part 6: Test the model

In [ ]:
#Fkt, die alle Zeilen des Testdatensatzes an den Prädiktor weitergibt und eine Vorhersage zurückgibt.

def get_prediction(row):
    prediction = round(float(predictor.predict(row[1:]))) #.decode('UTF-8')
    return prediction

#Prädiktion der ungesehenen Testdaten
with s3.open(f'{data_bucket}/{subfolder}/processed/test.csv') as f:
    test_data = pd.read_csv(f)
    
#Anwenden von get_prediction auf unsere Testdaten
cols = list(test_data.columns)
test_data['prediction'] = test_data.apply(get_prediction, axis=1)

#Erstellen einer neuen Tabelle mit den Prädiktionen
test_data = test_data[['prediction'] + cols]
test_data[:10]

In [ ]:
#Welche Prädiktionen weichen von der Grundwahrheit ab?


In [ ]:
#Erstellen einer Confusion Matrix


In [ ]:
#Berechnen der Genauigkeit (Accuracy)


In [ ]:
#Bestimmung des F1-Scores des Modells


## Remove the Endpoint (optional)
Comment out this cell to remove the endpoint if you want the endpoint to exist after "run all"

In [ ]:
#Löschen des Endpoints
sess.delete_endpoint(endpoint_name)